# Grape disease classification

Classify whether the grape or the leaf has a defect

### 1. Setup notebook

Imports, configurations

In [1]:
import os
import glob

import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [2]:
data_src = r'C:\data\grape_disease'

In [3]:
batch_size = 2
img_height = 224
img_width = 224

### 2. Load data

In [4]:
train_dir = os.path.join(data_src, 'train')
test_dir = os.path.join(data_src, 'test')

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # validation_split=0.2
) # set validation split

test_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    # validation_split=0.99
) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = test_datagen.flow_from_directory(
    test_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as validation data

Found 7222 images belonging to 4 classes.
Found 1805 images belonging to 4 classes.


In [16]:
exit_val = 10

for i, d in enumerate(validation_generator):
    print(d[0].shape)
    print(d[1].shape)

    if i > exit_val:
        break

(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)
(2, 224, 224, 3)
(2, 4)


### 3. Construct and train the model

In [6]:
base_model = tf.keras.applications.ResNet50V2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_height, img_width, 3),
)

for layer in base_model.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
predictions = tf.keras.layers.Dense(4, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
# head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [7]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
history = model.fit(train_generator, epochs=40, validation_data=validation_generator)

Epoch 1/40


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\steph\anaconda3\envs\ds_playground\lib\site-packages\tornad